In [ ]:
!pip install arxiv

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6089 sha256=7aadfac8c8c46092900ccf4b02eb52c5eb8d1eea73e49f69ca62ba4e1b8caa77
  Stored in directory: /home/codespace/.cache/pip/wheels/03/f5/1a/23761066dac1d0e8e683e5fdb27e12de53209d05a4a37e6246
Successfully built sgmllib3k

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [6]:
!pip install arxiv requests PyPDF2


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [1]:
from src.arxiv_data import fetch_all_years, save_data


In [10]:
# Fetch data for a specific range (e.g., 1991 to 1991, with PDF extraction)
arxiv_data_summary = fetch_all_years(start_year=1990, end_year=2025, month=3, download_pdf=False)

# Save the data to a JSON file for later analysis
save_data(arxiv_data_summary, "arxiv_march_data.json")


Fetching data for March 1990...
Query for March 1990: submittedDate:[19900301 TO 19900331]
  Found 2 papers.
Fetching data for March 1991...
Query for March 1991: submittedDate:[19910301 TO 19910331]
  Found 3 papers.
Fetching data for March 1992...
Query for March 1992: submittedDate:[19920301 TO 19920331]
  Found 117 papers.
Fetching data for March 1993...
Query for March 1993: submittedDate:[19930301 TO 19930331]
  Found 200 papers.
Fetching data for March 1994...
Query for March 1994: submittedDate:[19940301 TO 19940331]
  Found 200 papers.
Fetching data for March 1995...
Query for March 1995: submittedDate:[19950301 TO 19950331]
  Found 200 papers.
Fetching data for March 1996...
Query for March 1996: submittedDate:[19960301 TO 19960331]
  Found 200 papers.
Fetching data for March 1997...
Query for March 1997: submittedDate:[19970301 TO 19970331]
  Found 200 papers.
Fetching data for March 1998...
Query for March 1998: submittedDate:[19980301 TO 19980331]
  Found 200 papers.
Fetch

In [2]:
# Fetch data for a specific range (e.g., 1991 to 1991, with PDF extraction)
arxiv_data = fetch_all_years(start_year=2000, end_year=2000, month=3, download_pdf=True)

# Save the data to a JSON file for later analysis
save_data(arxiv_data, "arxiv_march_data_with_pdf.json")


Fetching data for March 2000...
Query for March 2000: submittedDate:[20000301 TO 20000331]
Error downloading PDF: HTTP 404
Error downloading PDF: HTTP 500
Error downloading PDF: HTTP 500
Error downloading PDF: HTTP 500


startxref on same line as offset
Illegal character in Name Object (b'/TJWCAJ+\xb7s\xb2\xd3\xa9\xfa\xc5\xe9')


Error extracting text from PDF: Unable to find 'endstream' marker for obj starting at 292176.
  Found 200 papers.
Data saved to arxiv_march_data_with_pdf.json


In [6]:
import requests
from io import BytesIO
from PyPDF2 import PdfReader
import re

def get_pdf_text(pdf_url):
    """
    Downloads the PDF from pdf_url, extracts its text using PyPDF2, 
    and performs basic cleanup on the extracted text.
    
    Returns an empty string if extraction fails.
    """
    try:
        response = requests.get(pdf_url)
        if response.status_code == 200:
            with BytesIO(response.content) as f:
                try:
                    pdf_reader = PdfReader(f)
                    text = ""
                    for page in pdf_reader.pages:
                        page_text = page.extract_text()
                        if page_text:
                            text += page_text + "\n"
                    
                    # Basic post-processing cleanup:
                    # 1. Replace common ligatures
                    text = text.replace("ﬁ", "fi").replace("ﬂ", "fl")
                    
                    # 2. Remove excessive newlines and whitespace
                    text = re.sub(r'\n\s*\n', '\n\n', text)
                    text = re.sub(r'\s+', ' ', text)
                    
                    return text.strip()
                except Exception as e:
                    print(f"Error extracting text from PDF: {e}")
                    return ""
        else:
            print(f"Error downloading PDF: HTTP {response.status_code}")
            return ""
    except Exception as e:
        print(f"Error during PDF request: {e}")
        return ""


In [4]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_url):
    import requests
    from io import BytesIO
    
    # Download PDF
    response = requests.get(pdf_url)
    if response.status_code != 200:
        print(f"Error: HTTP {response.status_code}")
        return ""
    
    pdf_data = BytesIO(response.content)
    doc = fitz.open(stream=pdf_data, filetype="pdf")
    
    full_text = ""
    for page in doc:
        full_text += page.get_text("text")
    
    return full_text

# Example usage:
pdf_url = "http://arxiv.org/pdf/hep-th/9203004v2"
text = extract_text_from_pdf(pdf_url)
print(text[:500])  # print first 500 characters for inspection


ModuleNotFoundError: No module named 'fitz'